# Modeling Pipeline (master_table)


In [68]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import joblib

try:
    from xgboost import XGBRegressor
except Exception as e:
    print("XGBoost no disponible, se omite:", e)


## 1. Data Cleaning & Inspection


In [69]:
# Cargar datos
data_path = Path("master_table.csv")
df = pd.read_csv(data_path)
print("Shape original:", df.shape)

# Filtrar filas con last_review_date == "unknown"
mask_unknown = df["last_review_date"].eq("unknown")
print("Filas con 'unknown' en last_review_date:", mask_unknown.sum())
df = df.loc[~mask_unknown].copy()
print("Shape tras filtrar unknown:", df.shape)

target_col = "cust_review_mean"

Shape original: (99441, 32)
Filas con 'unknown' en last_review_date: 768
Shape tras filtrar unknown: (98673, 32)



## 2. Preprocessing



In [70]:
cols = [
    "last_review_date",
    "customer_id",
    "log_cust_review_mean",
    "cust_review_std",
    "cust_num_good_reviews",
    "cust_num_bad_reviews"
]
# Extraer año/mes de la fecha y descartar la columna cruda
df["last_review_date"] = pd.to_datetime(df["last_review_date"], errors="coerce")
df["last_review_year"] = df["last_review_date"].dt.year
df["last_review_month"] = df["last_review_date"].dt.month

# Descartadno columnas en cols para prevenir posible lable leakage
df = df.drop(columns=cols)

> Las features en cols contienen información que se deriva, o que solo se conoce, del target que se esta tratando de predecir.
Especificamente con 'last_review_date' le da al modelo un atajo injusto y hace que parezca mejor durante el entrenamiento de lo que realmente será en la vida real.

In [71]:
# Separar target y features
X = df.drop(columns=[target_col])
y = df[target_col]

# Identificar columnas numéricas y categóricas
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
print("Numéricas:", numeric_cols)
print("Categóricas:", categorical_cols)

# Definir preprocesador (scaler + one-hot)
numeric_transformer = Pipeline([
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

Numéricas: ['quarter_neutral', 'semester_neutral', 'month_neutral', 'cust_avg_price_x_cust_installments_mean', 'cust_avg_freight', 'cust_avg_price', 'cust_avg_freight_x_cust_avg_delivery', 'cust_total_spent', 'cust_avg_freight_x_cust_installments_mean', 'cust_total_spent_x_cust_installments_mean', 'cust_avg_month', 'log_cust_total_spent', 'quarter_satisfecho', 'cust_late_ratio', 'cust_payment_methods', 'cust_num_products', 'cust_avg_freight_x_cust_late_ratio', 'cust_total_spent_x_cust_late_ratio', 'cust_payment_total', 'month_satisfecho', 'cust_avg_price_x_cust_avg_delivery', 'cust_total_orders', 'log_cust_avg_delivery', 'cust_avg_weekday', 'cust_avg_delivery']
Categóricas: []


### Train/Test Split

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
print("Train shape:", X_train.shape, "\nTest shape:", X_test.shape)


Train shape: (78938, 27) 
Test shape: (19735, 27)


## 3. Model Training & Tuning (Regresión)
Para cada modelo:
1. Pipeline con preprocesador.
2. Grilla de hiperparámetros (3 variaciones por hiperparámetro).
3. GridSearchCV con cv=5 y dos métricas (MSE y R2) en el conjunto de entrenamiento.
4. Guardar métricas de CV y el mejor estimador.


In [73]:
results = []
best_model = None
best_model_name = None
best_mse = np.inf


### 3.1 Linear Regression


In [74]:
lr_pipe = Pipeline([
    ("prep", preprocessor),
    ("model", LinearRegression())
])

lr_grid = {
    "model__fit_intercept": [True, False, True],
    "model__n_jobs": [None, -1, 1]
}

lr_search = GridSearchCV(
    estimator=lr_pipe,
    param_grid=lr_grid,
    cv=5,
    scoring={"mse": "neg_mean_squared_error", "r2": "r2"},
    refit="mse",
    n_jobs=-1
)
lr_search.fit(X_train, y_train)
best_idx = lr_search.best_index_
lr_mse_cv = -lr_search.cv_results_["mean_test_mse"][best_idx]
lr_r2_cv = lr_search.cv_results_["mean_test_r2"][best_idx]
lr_best = lr_search.best_estimator_
print("LR mejores params:", lr_search.best_params_)
print(f"LR CV (cv=5) MSE: {lr_mse_cv:.4f}, R2: {lr_r2_cv:.4f}")

results.append({
    "model": "LinearRegression",
    "mse": lr_mse_cv,
    "r2": lr_r2_cv,
    "best_params": lr_search.best_params_
})
if lr_mse_cv < best_mse:
    best_mse = lr_mse_cv
    best_model = lr_best
    best_model_name = "LinearRegression"


LR mejores params: {'model__fit_intercept': True, 'model__n_jobs': None}
LR CV (cv=5) MSE: 0.6525, R2: 0.6401


### 3.2 RandomForestRegressor


In [75]:
rf_pipe = Pipeline([
    ("prep", preprocessor),
    ("model", RandomForestRegressor(random_state=42, n_jobs=-1))
])

rf_grid = {
    "model__n_estimators": [100, 200, 300],
    "model__max_depth": [None, 10, 20]
}

rf_search = GridSearchCV(
    estimator=rf_pipe,
    param_grid=rf_grid,
    cv=5,
    scoring={"mse": "neg_mean_squared_error", "r2": "r2"},
    refit="mse",
    n_jobs=-1
)
rf_search.fit(X_train, y_train)
best_idx = rf_search.best_index_
rf_mse_cv = -rf_search.cv_results_["mean_test_mse"][best_idx]
rf_r2_cv = rf_search.cv_results_["mean_test_r2"][best_idx]
rf_best = rf_search.best_estimator_
print("RF mejores params:", rf_search.best_params_)
print(f"RF CV (cv=5) MSE: {rf_mse_cv:.4f}, R2: {rf_r2_cv:.4f}")

results.append({
    "model": "RandomForest",
    "mse": rf_mse_cv,
    "r2": rf_r2_cv,
    "best_params": rf_search.best_params_
})
if rf_mse_cv < best_mse:
    best_mse = rf_mse_cv
    best_model = rf_best
    best_model_name = "RandomForest"


RF mejores params: {'model__max_depth': 10, 'model__n_estimators': 300}
RF CV (cv=5) MSE: 0.1692, R2: 0.9067


### 3.3 GradientBoostingRegressor


In [76]:
gb_pipe = Pipeline([
    ("prep", preprocessor),
    ("model", GradientBoostingRegressor(random_state=42))
])

gb_grid = {
    "model__n_estimators": [100, 200, 300],
    "model__learning_rate": [0.05, 0.1, 0.2]
}

gb_search = GridSearchCV(
    estimator=gb_pipe,
    param_grid=gb_grid,
    cv=5,
    scoring={"mse": "neg_mean_squared_error", "r2": "r2"},
    refit="mse",
    n_jobs=-1
)
gb_search.fit(X_train, y_train)
best_idx = gb_search.best_index_
gb_mse_cv = -gb_search.cv_results_["mean_test_mse"][best_idx]
gb_r2_cv = gb_search.cv_results_["mean_test_r2"][best_idx]
gb_best = gb_search.best_estimator_
print("GB mejores params:", gb_search.best_params_)
print(f"GB CV (cv=5) MSE: {gb_mse_cv:.4f}, R2: {gb_r2_cv:.4f}")

results.append({
    "model": "GradientBoosting",
    "mse": gb_mse_cv,
    "r2": gb_r2_cv,
    "best_params": gb_search.best_params_
})
if gb_mse_cv < best_mse:
    best_mse = gb_mse_cv
    best_model = gb_best
    best_model_name = "GradientBoosting"


GB mejores params: {'model__learning_rate': 0.2, 'model__n_estimators': 100}
GB CV (cv=5) MSE: 0.1689, R2: 0.9069


### 3.4 XGBRegressor (opcional si está instalado)


In [77]:

xgb_pipe = Pipeline([
    ("prep", preprocessor),
    ("model", XGBRegressor(random_state=42, n_jobs=-1, objective="reg:squarederror"))
])

xgb_grid = {
    "model__n_estimators": [200, 400, 600],
    "model__max_depth": [4, 6, 8],
    "model__learning_rate": [0.05, 0.1, 0.2]
}

xgb_search = GridSearchCV(
    estimator=xgb_pipe,
    param_grid=xgb_grid,
    cv=5,
    scoring={"mse": "neg_mean_squared_error", "r2": "r2"},
    refit="mse",
    n_jobs=-1
)
xgb_search.fit(X_train, y_train)
best_idx = xgb_search.best_index_
xgb_mse_cv = -xgb_search.cv_results_["mean_test_mse"][best_idx]
xgb_r2_cv = xgb_search.cv_results_["mean_test_r2"][best_idx]
xgb_best = xgb_search.best_estimator_
print("XGB mejores params:", xgb_search.best_params_)
print(f"XGB CV (cv=5) MSE: {xgb_mse_cv:.4f}, R2: {xgb_r2_cv:.4f}")

results.append({
    "model": "XGBoost",
    "mse": xgb_mse_cv,
    "r2": xgb_r2_cv,
    "best_params": xgb_search.best_params_
})
if xgb_mse_cv < best_mse:
    best_mse = xgb_mse_cv
    best_model = xgb_best
    best_model_name = "XGBoost"

/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


XGB mejores params: {'model__learning_rate': 0.1, 'model__max_depth': 4, 'model__n_estimators': 200}
XGB CV (cv=5) MSE: 0.1681, R2: 0.9073


## 4. Evaluation & Selection


In [81]:
results_df = pd.DataFrame(results)
print("Resultados GridSearchCV (5CV):")
display(results_df)

# best_model ya viene refiteado en train por GridSearchCV (refit="mse")

# Evaluar en test
y_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print("\nResultados Test Set:")
print("Mejor modelo:", best_model_name)
print(f"MSE CV (cv=5): {best_mse:.4f}")
print(f"MSE en test holdout: {test_mse:.4f}")
print(f"R2 en test holdout: {test_r2:.4f}")

# Guardar el mejor modelo (incluye preprocesador)
joblib.dump(best_model, "best_model.pkl")
print("Modelo guardado en best_model.pkl")


Resultados GridSearchCV (5CV):


,model,mse,r2,best_params
0,LinearRegression,0.652456,0.640082,"{'model__fit_intercept': True, 'model__n_jobs'..."
1,RandomForest,0.169207,0.906694,"{'model__max_depth': 10, 'model__n_estimators'..."
2,GradientBoosting,0.168888,0.906869,"{'model__learning_rate': 0.2, 'model__n_estima..."
3,XGBoost,0.168058,0.907327,"{'model__learning_rate': 0.1, 'model__max_dept..."



Resultados Test Set:
Mejor modelo: XGBoost
MSE CV (cv=5): 0.1681
MSE en test holdout: 0.1684
R2 en test holdout: 0.9068
Modelo guardado en best_model.pkl
